In [1]:
import sys
sys.path.append('../')
from msBO import MultiStateBO
from msBO.objective import BPMvar_minimization
sys.path.append('../../machineIO/')
from machineIO import construct_machineIO, StatefulOracleEvaluator
from machineIO.preset import get_limits

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
from epics import caget, caget_many, caput_many
from phantasy import fetch_data
import datetime
from phantasy import ensure_set, fetch_data

# import importlib, msBO  # or from boom import msBO if that's your import
# importlib.reload(msBO)

Failed to import 'phantasy'


ModuleNotFoundError: No module named 'machineIO.preset'

In [2]:
n_init = 8
n_each = 2
n_states = 3
n_iter = 3

In [3]:
SCS = caget("ACS_DIAG:DEST:ACTIVE_ION_SOURCE")
ion = caget("FE_ISRC"+str(SCS)+":BEAM:ELMT_BOOK")
Q = caget("FE_ISRC"+str(SCS)+":BEAM:Q_BOOK")
A = caget("FE_ISRC"+str(SCS)+":BEAM:A_BOOK")
# AQ = caget("FE_ISRC2:BEAM:MOVRQ_BOOK")
AQ = A/Q
ion = str(A)+ion+str(Q)
print('SCS'+str(SCS), ion, 'A/Q=',AQ)

SCS2 64Zn19 A/Q= 3.3684210526315788


In [4]:
now0 = datetime.datetime.now()
fname = now0.strftime('%Y%m%d_%H%M')+'['+ion+'][msBO]test'
fname

'20251112_0730[64Zn19][msBO]test'

### construct machineIO

In [5]:
mIO = construct_machineIO(test=True)

In [ ]:
mIO.__dict__

In [ ]:
ramping_time = 1
BO_comp_time = 10
dt = mIO._ensure_set_timewait_after_ramp + mIO._fetch_data_time_span + ramping_time + BO_comp_time
print(f'expected run time: {int((n_init*n_state + n_iter*n_state*n_each)*dt/60)} min')
print(f'expected number of data: {n_init*n_state + n_iter*n_state*n_each}')

### control knobs

In [6]:
control_CSETs = [
    'BDS_BTS:PSC2_D5496:I_CSET',
    'BDS_BTS:PSC1_D5496:I_CSET',
    'BDS_BTS:PSC2_D5563:I_CSET',
    'BDS_BTS:PSC1_D5563:I_CSET',
#     'BDS_BBS:PSC2_D5616:I_CSET',
#     'BDS_BBS:PSC1_D5616:I_CSET',
#     'BDS_BBS:PSC2_D5657:I_CSET',
#     'BDS_BBS:PSC1_D5657:I_CSET',
#     'BDS_BBS:PSC2_D5693:I_CSET',
#     'BDS_BBS:PSC1_D5693:I_CSET',
    'BDS_BBS:PSQ_D5611:I_CSET',
    'BDS_BBS:PSS_D5606:I_CSET',
    'BDS_FFS:PSC2_D5746:I_CSET',
    'BDS_FFS:PSC1_D5746:I_CSET',
]
control_RDs = [pv.replace('CSET','RD') for pv in control_CSETs]

control_ref = caget_many(control_CSETs)
control_rd_ref = caget_many(control_RDs)
control_tols_ref = 2*np.abs(control_ref - control_rd_ref)

control_tols = []
control_min = []
control_max = []
i = 0
for v, PV in zip(control_ref,control_CSETs):
    if ':PSC' in PV:
        control_min.append(v - 3)
        control_max.append(v + 3)
        control_tols.append(max(0.1,control_tols_ref[i]))
    elif ':PSQ_' in PV:
        control_min.append(v*0.95)
        control_max.append(v*1.05)
        control_tols.append(max(0.1,control_tols_ref[i]))
    elif ':PSS_' in PV:
        control_min.append(v*0.3)
        control_max.append(v*1.7)
        control_tols.append(max(0.05,control_tols_ref[i]))
    else:
        raise ValueError(f'control min/max cannot be determined automatically for {PV}')
    i+=1
        
        
        
##== Manually set decision bounds and tolerance
# control_min = [ -5, -5, -5, -5]
# control_max = [  5,  5,  5,  5]
# control_tols = [0.2,0.2,0.2,0.2]

assert len(control_CSETs) == len(control_min) == len(control_max) == len(control_tols)
control_Lo_limit, control_Hi_limit = get_limits(control_CSETs)
control_min = np.clip(control_min, a_min = control_Lo_limit, a_max = None)
control_max = np.clip(control_max, a_min = None, a_max = control_Hi_limit)
assert np.all(control_max > control_min)

pd.DataFrame(np.array([control_ref,control_min,control_max,control_tols,control_Lo_limit,control_Hi_limit]).T,
             index=control_CSETs, 
             columns=['current value','decision min','decision max','tol','LoLim','HiLim'])

,current value,decision min,decision max,tol,LoLim,HiLim
BDS_BTS:PSC2_D5496:I_CSET,2.085600,-0.914400,5.085600,0.10,-51.4,51.4
BDS_BTS:PSC1_D5496:I_CSET,6.141800,3.141800,9.141800,0.10,-51.4,51.4
BDS_BBS:PSC1_D5616:I_CSET,8.744000,5.744000,11.744000,0.10,-186.0,186.0
BDS_BBS:PSC1_D5657:I_CSET,0.000000,-3.000000,3.000000,0.10,-186.0,186.0
BDS_BBS:PSC1_D5693:I_CSET,0.310600,-2.689400,3.310600,0.10,-186.0,186.0
BDS_BBS:PSQ_D5611:I_CSET,133.225122,126.563866,139.886378,0.15,0.0,309.0
BDS_BBS:PSS_D5606:I_CSET,0.414000,0.082800,0.745200,0.05,0.0,4.8


In [7]:
control_couplings = {
    'BDS_BBS:PSQ_D5611:I_CSET':
        {
            'CSETs' :['BDS_BBS:PSQ_D5621:I_CSET','BDS_BBS:PSQ_D5688:I_CSET','BDS_BBS:PSQ_D5698:I_CSET'],
            'RDs'   :[],
            'tols'  :[],
            'coeffs':[],
        },
     'BDS_BBS:PSS_D5606:I_CSET':
        {
            'CSETs' :['BDS_BBS:PSS_D5703:I_CSET'],
            'RDs'   :[],
            'tols'  :[],
            'coeffs':[],
        },
}

for pv, val in control_couplings.items():
    x = caget(pv)
    y = np.array(caget_many(val['CSETs']))
    ipv = control_CSETs.index(pv)
    tol = control_tols[ipv]
    val['RDs'] = [pv.replace('CSET','RD') for pv in val['CSETs']]
    val['tols'] = [tol]*len(val['CSETs'])
    val['coeffs'] = y/x
    
control_couplings

{'BDS_BBS:PSQ_D5611:I_CSET': {'CSETs': ['BDS_BBS:PSQ_D5621:I_CSET',
   'BDS_BBS:PSQ_D5688:I_CSET',
   'BDS_BBS:PSQ_D5698:I_CSET'],
  'RDs': ['BDS_BBS:PSQ_D5621:I_RD',
   'BDS_BBS:PSQ_D5688:I_RD',
   'BDS_BBS:PSQ_D5698:I_RD'],
  'tols': [0.15, 0.15, 0.15],
  'coeffs': array([0.59426309, 0.59426309, 1.        ])},
 'BDS_BBS:PSS_D5606:I_CSET': {'CSETs': ['BDS_BBS:PSS_D5703:I_CSET'],
  'RDs': ['BDS_BBS:PSS_D5703:I_RD'],
  'tols': [0.05],
  'coeffs': array([1.00089638])}}

### state definition

In [13]:
state_CSETs  = ['FS1_BBS:CSEL_D2405:CTR_MTR.VAL']
state_RDs    = ['FS1_BBS:CSEL_D2405:CTR_MTR.RBV']
state_tols   = [0.1]*len(state_CSETs)
state_val0   = caget_many(state_CSETs)
state_key_vals  = {'28+':[10],
                   '29+':[20]}
states   = list(state_key_vals.keys())
n_states = len(states)
state_key_vals

{'q28': [10], 'q29': [20]}

### monitors

In [14]:
monitor_BPMs = ['BDS_FFS:BPM_D5742','BDS_FFS:BPM_D5772','BDS_FFS:BPM_D5790']
monitor_RDs = []
for bpm in monitor_BPMs:
    monitor_RDs += [bpm+':XPOS_RD',bpm+':YPOS_RD',bpm+':MAG_RD']

### Define Oracle 
# WARN!  This cell may change state!

In [15]:
bpm_posRDs = [pv for pv in monitor_RDs if 'POS_RD' in pv]
bpm_magRDs = [pv for pv in monitor_RDs if 'MAG_RD' in pv]

BPM_MAGs_ref = {}
for state, goal in state_key_vals.items():
    input(f'set {state_CSETs} to {goal}')
#     ret = ensure_set(state_CSETs,state_RDs,goal,state_tols,timeout=30)
    val, _ = fetch_data(bpm_magRDs,5)
    BPM_MAGs_ref[state] = val
BPM_MAGs_ref

{'q28': array([0.00089236, 0.00081054, 0.00091569]),
 'q29': array([0.00030794, 0.00027993, 0.00031815])}

In [16]:
def BPM_MAG_obj(df,s):
    df['BPM:MAG_min_ratio'] = (df[bpm_magRDs].values/BPM_MAGs_ref[s][None,:]).min()
    return df

In [17]:
oracle_key_names = {'x':control_RDs,
                    'y':bpm_posRDs + ['BPM:MAG_min_ratio']}
oracle_key_names

{'x': ['BDS_BTS:PSC2_D5496:I_RD',
  'BDS_BTS:PSC1_D5496:I_RD',
  'BDS_BBS:PSC1_D5616:I_RD',
  'BDS_BBS:PSC1_D5657:I_RD',
  'BDS_BBS:PSC1_D5693:I_RD',
  'BDS_BBS:PSQ_D5611:I_RD',
  'BDS_BBS:PSS_D5606:I_RD'],
 'y': ['BDS_FFS:BPM_D5742:XPOS_RD',
  'BDS_FFS:BPM_D5742:YPOS_RD',
  'BDS_FFS:BPM_D5772:XPOS_RD',
  'BDS_FFS:BPM_D5772:YPOS_RD',
  'BDS_FFS:BPM_D5790:XPOS_RD',
  'BDS_FFS:BPM_D5790:YPOS_RD',
  'BPM:MAG_min_ratio']}

In [18]:
oracleEvaluator = StatefulOracleEvaluator(
    mIO,
    control_CSETs= control_CSETs,
    control_RDs  = control_RDs,
    control_tols = control_tols,
    control_couplings = control_couplings,
    state_CSETs  = state_CSETs,
    state_RDs    = state_RDs,
    state_tols   = state_tols,
    state_key_vals = state_key_vals,
    oracle_key_names = oracle_key_names,
    monitor_RDs  = monitor_RDs,
    state_df_manipulators = [BPM_MAG_obj],
)

# Define Composite Object

In [19]:
n_state = len(states)
n_task = len(bpm_posRDs)+1
composite_objective_function = BPMvar_minimization(S=n_state, J=n_task)

# Define msBO

In [20]:
local_optimization = False
acq_type = 'EI'
fix_acq_state = False

msbo = MultiStateBO(
    states = states,
    tasks  = oracle_key_names['y'], 
    control_min = control_min,
    control_max = control_max,
    multistate_oracle_evaluator = oracleEvaluator,
    composite_objective_function = composite_objective_function,
    local_bound_size = 0.2*(np.asarray(control_max) - np.asarray(control_min))
    )

pvlist ['BDS_FFS:BPM_D5742:XPOS_RD', 'BDS_FFS:BPM_D5742:YPOS_RD', 'BDS_BBS:PSS_D5606:I_RD', 'BDS_BBS:PSQ_D5688:I_CSET', 'BDS_BTS:PSC1_D5496:I_CSET', 'BDS_BBS:PSC1_D5616:I_RD', 'BDS_BBS:PSQ_D5611:I_RD', 'BDS_FFS:BPM_D5790:MAG_RD', 'BDS_BBS:PSS_D5703:I_RD', 'BDS_BBS:PSS_D5606:I_CSET', 'BDS_BBS:PSQ_D5611:I_CSET', 'BDS_BBS:PSQ_D5621:I_RD', 'BDS_BTS:PSC2_D5496:I_CSET', 'BDS_BBS:PSC1_D5693:I_CSET', 'BDS_BBS:PSQ_D5698:I_CSET', 'BDS_FFS:BPM_D5742:MAG_RD', 'BDS_BBS:PSC1_D5657:I_RD', 'BDS_BTS:PSC1_D5496:I_RD', 'BDS_FFS:BPM_D5790:XPOS_RD', 'BDS_BBS:PSS_D5703:I_CSET', 'BDS_BBS:PSC1_D5693:I_RD', 'BDS_BBS:PSQ_D5621:I_CSET', 'BDS_BBS:PSQ_D5688:I_RD', 'BDS_BBS:PSC1_D5657:I_CSET', 'BDS_BBS:PSC1_D5616:I_CSET', 'BDS_BBS:PSQ_D5698:I_RD', 'FS1_BBS:CSEL_D2405:CTR_MTR.RBV', 'BDS_FFS:BPM_D5772:MAG_RD', 'FS1_BBS:CSEL_D2405:CTR_MTR.VAL', 'BDS_BTS:PSC2_D5496:I_RD', 'BDS_FFS:BPM_D5790:YPOS_RD', 'BDS_FFS:BPM_D5772:XPOS_RD', 'BDS_FFS:BPM_D5772:YPOS_RD']
len(pvlist),len(set(pvlist)) 33 33
[07:30:19.185] WARNING: pha

# run msBO

In [21]:
msbo.init(n_init=n_init, local_optimization=local_optimization)
for i_iter in range(n_iter):
    if i_iter%2==0:
        states_order = states[::-1]
    else:
        states_order = states
    for s in states_order:
        print(i_iter, s)
        for i_each in range(n_each):
            msbo.step(s=s, local_optimization=local_optimization, acq_type=acq_type, fix_acq_state=fix_acq_state)

pvlist ['BDS_FFS:BPM_D5742:XPOS_RD', 'BDS_FFS:BPM_D5742:YPOS_RD', 'BDS_BBS:PSS_D5606:I_RD', 'BDS_BBS:PSQ_D5688:I_CSET', 'BDS_BTS:PSC1_D5496:I_CSET', 'BDS_BBS:PSC1_D5616:I_RD', 'BDS_BBS:PSQ_D5611:I_RD', 'BDS_FFS:BPM_D5790:MAG_RD', 'BDS_BBS:PSS_D5703:I_RD', 'BDS_BBS:PSS_D5606:I_CSET', 'BDS_BBS:PSQ_D5611:I_CSET', 'BDS_BBS:PSQ_D5621:I_RD', 'BDS_BTS:PSC2_D5496:I_CSET', 'BDS_BBS:PSC1_D5693:I_CSET', 'BDS_BBS:PSQ_D5698:I_CSET', 'BDS_FFS:BPM_D5742:MAG_RD', 'BDS_BBS:PSC1_D5657:I_RD', 'BDS_BTS:PSC1_D5496:I_RD', 'BDS_FFS:BPM_D5790:XPOS_RD', 'BDS_BBS:PSS_D5703:I_CSET', 'BDS_BBS:PSC1_D5693:I_RD', 'BDS_BBS:PSQ_D5621:I_CSET', 'BDS_BBS:PSQ_D5688:I_RD', 'BDS_BBS:PSC1_D5657:I_CSET', 'BDS_BBS:PSC1_D5616:I_CSET', 'BDS_BBS:PSQ_D5698:I_RD', 'FS1_BBS:CSEL_D2405:CTR_MTR.RBV', 'BDS_FFS:BPM_D5772:MAG_RD', 'FS1_BBS:CSEL_D2405:CTR_MTR.VAL', 'BDS_BTS:PSC2_D5496:I_RD', 'BDS_FFS:BPM_D5790:YPOS_RD', 'BDS_FFS:BPM_D5772:XPOS_RD', 'BDS_FFS:BPM_D5772:YPOS_RD']
len(pvlist),len(set(pvlist)) 33 33
[07:30:21.449] WARNING: pha

self.states ['q28', 'q29']
oracle["state"] ['q28']
pvlist ['BDS_FFS:BPM_D5742:XPOS_RD', 'BDS_FFS:BPM_D5742:YPOS_RD', 'BDS_BBS:PSS_D5606:I_RD', 'BDS_BBS:PSQ_D5688:I_CSET', 'BDS_BTS:PSC1_D5496:I_CSET', 'BDS_BBS:PSC1_D5616:I_RD', 'BDS_BBS:PSQ_D5611:I_RD', 'BDS_FFS:BPM_D5790:MAG_RD', 'BDS_BBS:PSS_D5703:I_RD', 'BDS_BBS:PSS_D5606:I_CSET', 'BDS_BBS:PSQ_D5611:I_CSET', 'BDS_BBS:PSQ_D5621:I_RD', 'BDS_BTS:PSC2_D5496:I_CSET', 'BDS_BBS:PSC1_D5693:I_CSET', 'BDS_BBS:PSQ_D5698:I_CSET', 'BDS_FFS:BPM_D5742:MAG_RD', 'BDS_BBS:PSC1_D5657:I_RD', 'BDS_BTS:PSC1_D5496:I_RD', 'BDS_FFS:BPM_D5790:XPOS_RD', 'BDS_BBS:PSS_D5703:I_CSET', 'BDS_BBS:PSC1_D5693:I_RD', 'BDS_BBS:PSQ_D5621:I_CSET', 'BDS_BBS:PSQ_D5688:I_RD', 'BDS_BBS:PSC1_D5657:I_CSET', 'BDS_BBS:PSC1_D5616:I_CSET', 'BDS_BBS:PSQ_D5698:I_RD', 'FS1_BBS:CSEL_D2405:CTR_MTR.RBV', 'BDS_FFS:BPM_D5772:MAG_RD', 'FS1_BBS:CSEL_D2405:CTR_MTR.VAL', 'BDS_BTS:PSC2_D5496:I_RD', 'BDS_FFS:BPM_D5790:YPOS_RD', 'BDS_FFS:BPM_D5772:XPOS_RD', 'BDS_FFS:BPM_D5772:YPOS_RD']
len(pvlist)

self.states ['q28', 'q29']
oracle["state"] ['q28']
set FS1_BBS:CSEL_D2405:CTR_MTR.VAL to 20.0
pvlist ['BDS_FFS:BPM_D5742:XPOS_RD', 'BDS_FFS:BPM_D5742:YPOS_RD', 'BDS_BBS:PSS_D5606:I_RD', 'BDS_BBS:PSQ_D5688:I_CSET', 'BDS_BTS:PSC1_D5496:I_CSET', 'BDS_BBS:PSC1_D5616:I_RD', 'BDS_BBS:PSQ_D5611:I_RD', 'BDS_FFS:BPM_D5790:MAG_RD', 'BDS_BBS:PSS_D5703:I_RD', 'BDS_BBS:PSS_D5606:I_CSET', 'BDS_BBS:PSQ_D5611:I_CSET', 'BDS_BBS:PSQ_D5621:I_RD', 'BDS_BTS:PSC2_D5496:I_CSET', 'BDS_BBS:PSC1_D5693:I_CSET', 'BDS_BBS:PSQ_D5698:I_CSET', 'BDS_FFS:BPM_D5742:MAG_RD', 'BDS_BBS:PSC1_D5657:I_RD', 'BDS_BTS:PSC1_D5496:I_RD', 'BDS_FFS:BPM_D5790:XPOS_RD', 'BDS_BBS:PSS_D5703:I_CSET', 'BDS_BBS:PSC1_D5693:I_RD', 'BDS_BBS:PSQ_D5621:I_CSET', 'BDS_BBS:PSQ_D5688:I_RD', 'BDS_BBS:PSC1_D5657:I_CSET', 'BDS_BBS:PSC1_D5616:I_CSET', 'BDS_BBS:PSQ_D5698:I_RD', 'FS1_BBS:CSEL_D2405:CTR_MTR.RBV', 'BDS_FFS:BPM_D5772:MAG_RD', 'FS1_BBS:CSEL_D2405:CTR_MTR.VAL', 'BDS_BTS:PSC2_D5496:I_RD', 'BDS_FFS:BPM_D5790:YPOS_RD', 'BDS_FFS:BPM_D5772:XPOS_RD

KeyboardInterrupt: 

### visualize optimization result

In [ ]:
fig,ax, virtualObjMean = msbo.plot_composite_objective()
ax[0].set_ylim(0.97,1)

In [ ]:
fig, axes = msbo.plot_state_predictions_history()

### set to best solution

In [ ]:
x_best = msbo.history['predictions'][-1]['x_best']
caput_many(control_CSETs,x_best)

### restore states

In [ ]:
caput_many(state_CSETs,state_val0)

### restore control if optimization result is not good

In [ ]:
# caput_many(control_CSETs,control_ref)